# Sparse stochastic variational inference

In [2]:
from tqdm import tqdm_notebook as tqdm
import autograd.numpy as np

# https://github.com/bamos/opt.py/blob/master/bamos_opt/pgd.py
def psgd(x0, g, proj, eps, maxit, lam, callback=None, **kwargs):
    """TODO."""
    results = {
        'feval': 0,
        'geval': 0
    }

    x = proj(np.copy(x0))

    for k in tqdm(range(maxit)):
        g_x = g(x, **kwargs)
        lam_val = lam(k)
        x = proj(x - lam_val * g_x)
        
        if callback:
            callback(k, x, g_x, lam_val)

    return x

def lam_inverse(init_lam):
    return lambda k: init_lam/k

def sort_rows(z):
    perm = anp.argsort(z, axis=1)
    rows = anp.arange(z.shape[0], dtype=int).reshape(-1, 1)
    return z[rows, perm]

def sparsemax_rows(z):
    ncols = z.shape[1]
    u = sort_rows(z)[:, ::-1]
    cssv = u.cumsum(axis=1) - 1
    ind = anp.arange(ncols) + 1
    cond = u - cssv / ind > 0
    argmin = anp.argmin(cond, axis=1)
    rho = np.where(argmin > 0, argmin, ncols)
    theta = cssv[anp.arange(z.shape[0]) , rho - 1]/rho
    return np.clip(z - theta.reshape(-1, 1), 0, None)

In [43]:
from autograd.scipy.stats import norm

n_clusters = 3
n_data = 4
mu = np.arange(n_clusters)
p = np.array([0.2, 0.3, 0.5])
indices = np.arange(n_clusters)
assignments = np.random.choice(indices, size=n_data, p=p)
x = np.random.normal(loc=mu[assignments])
log_p = lambda assignments: np.sum(norm.logpdf(x, loc=mu[assignments]) + np.log(p[assignments]))
log_p(assignments), log_p(np.ones_like(assignments))

(-9.638855789146623, -12.45632007485198)

In [45]:
log_q = lambda assignments, p: np.log(p[assignments]).sum()
q0 = np.ones((n_data, n_clusters))/n_clusters
q0

array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])